# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
city_df = pd.read_csv('city_data.csv')
city_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hermanus,-34.4187,19.2345,54.81,86,1,6.29,ZA,1661145301
1,1,Chumikan,54.7183,135.3150,57.47,73,100,15.12,RU,1661111179
2,2,Begūn,24.9833,75.0000,73.53,94,81,7.40,IN,1661128616
3,3,Avarua,-21.2078,-159.7750,75.25,88,100,14.97,CK,1661101057
4,4,Nadym,65.5333,72.5167,49.75,84,100,2.80,RU,1661123626


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(city_df[["Lat", "Lng"]], weights=city_df["Humidity"], max_intensity=100)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
hotel_df = city_df.loc[(city_df["Max Temp"] < 80) & (city_df["Max Temp"] > 70) & (city_df["Wind Speed"] < 10) & \
                          (city_df["Cloudiness"] == 0)]
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
269,269,Stevens Point,44.5236,-89.5746,79.47,45,0,7.00,US,1661080095
310,310,Ajdabiya,30.7554,20.2263,78.53,77,0,6.82,LY,1661141379
334,334,Constantine,36.3650,6.6147,72.81,53,0,2.30,DZ,1661144220
361,361,Creston,41.0586,-94.3614,78.76,57,0,5.75,US,1661081599
367,367,Natal,-5.7950,-35.2094,75.85,88,0,6.91,BR,1661070346


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
# Geocoordinates.
sample_coordinate = "44.5236,-89.5746"
target_search = "Hotels"
target_radius = 5000
target_type = "lodging"

# Setting up a parameters dictionary. 
my_params = {"location": sample_coordinate, "keyword": target_search, "radius": target_radius, "type": target_type, \
                "key": g_key}

# Requesting the hotels from a Google Maps URL.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=my_params)

In [33]:
data = response.json()
print(json.dumps(data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 44.5248323,
                    "lng": -89.5829935
                },
                "viewport": {
                    "northeast": {
                        "lat": 44.52632622989272,
                        "lng": -89.58174427010728
                    },
                    "southwest": {
                        "lat": 44.52362657010728,
                        "lng": -89.58444392989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Cobblestone Hotel & Suites - Stevens Point",
            "opening_hours": {
                "op

In [36]:
print(data["results"][0]["name"])

Cobblestone Hotel & Suites - Stevens Point


In [37]:
hotel_names = data(["results"][0]["name"])
hotel_df["Hotel Name"] = hotel_names
hotel_df.head()

TypeError: string indices must be integers

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
